# Explain code and suggest code with llama7b

This notebook uses a quantized version of LLama 7B.

## Install dependencies and prepare helper code

In [1]:
# https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF

!CT_CUBLAS=1 pip install ctransformers --no-binary ctransformers
!pip install --quiet accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ctransformers: filename=ctransformers-0.2.27-cp310-cp310-linux_x86_64.whl size=2336834 sha256=3a4bc336834f8fa4f3e24e3ec6d7ebcf33b83d3f6b5b1b97a4a2a1af0cddc0c0
  Stored in directory: /root/.cache/pip/wheels/dd/54/e9/32364da8eee84a2b0b412394983c15add18816c507e90f02d8
Successfully built ctransformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 61.2 MB/s eta 0:00:00


In [2]:
# check GPU status
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jul 17 08:39:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import torch

def get_gpu_memory_usage():
  """
  Returns the current free and used memory of the GPU in megabytes.
  """
  device = torch.cuda.current_device()
  properties = torch.cuda.get_device_properties(device)
  free_memory, total_memory = torch.cuda.mem_get_info(device)
  used_memory = total_memory - free_memory
  return free_memory / 1024**2, used_memory / 1024**2

free_memory, used_memory = get_gpu_memory_usage()
print(f"Free memory: {free_memory:.2f} MB")
print(f"Used memory: {used_memory:.2f} MB")
print(torch.cuda.current_device())

Free memory: 14999.06 MB
Used memory: 103.00 MB
0


Load model.

In [4]:
from ctransformers import AutoModelForCausalLM, AutoConfig


config = AutoConfig.from_pretrained("TheBloke/Llama-2-7b-Chat-GGUF")
config.max_seq_len = 2048 #4096
config.max_answer_len= 1024

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = AutoModelForCausalLM.from_pretrained(
    "TheBloke/Llama-2-7b-Chat-GGUF",
    model_file="llama-2-7b-chat.Q5_K_S.gguf",
    # model_file="llama-2-7b-chat.Q4_0.gguf",
    # model_file="llama-2-7b-chat.Q6_K.gguf",
    model_type="llama",
    gpu_layers=130,
    context_length=4096,
    max_new_tokens=2048,
  )



Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama-2-7b-chat.Q5_K_S.gguf:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

In [5]:
# check that the context length has been set
llm.context_length

4096

Render helpers

In [6]:
from IPython.display import display, Markdown

def as_md(text):
  return display(Markdown(text))

In [7]:
import markdown
import ipywidgets as widgets
from IPython.display import display
from typing import Callable

AnswerSetter = Callable[[str], None]

spinner = """
<div class="spinner" style="display:inline-block;position:relative;width:80px;height:40px;">
  <div
    style="display:inline-block;position:absolute;left:8px;width:16px;height:16px;border-radius:50%;background-color:#999;animation:bounce 2s infinite ease-in-out;">
  </div>
  <div
    style="display:inline-block;position:absolute;left:32px;width:16px;height:16px;border-radius:50%;background-color:#999;animation:bounce 2s infinite ease-in-out;animation-delay:-0.2s;">
  </div>
  <div
    style="display:inline-block;position:absolute;left:56px;width:16px;height:16px;border-radius:50%;background-color:#999;animation:bounce 2s infinite ease-in-out;animation-delay:-0.4s;">
  </div>
</div>
<style>
  .spinner {
    display: inline-block;
    position: relative;
    width: 80px;
    height: 80px;
  }

  .spinner div {
    display: inline-block;
    position: absolute;
    left: 8px;
    width: 16px;
    height: 16px;
    border-radius: 50%;
    background-color: #333;
    animation: bounce 2s infinite ease-in-out;
  }

  .spinner div:nth-child(2) {
    left: 32px;
    animation-delay: -0.2s;
  }

  .spinner div:nth-child(3) {
    left: 56px;
    animation-delay: -0.4s;
  }

  @keyframes bounce {

    0%,
    80%,
    100% {
      transform: scale(0);
    }

    40% {
      transform: scale(1);
    }
  }
</style>
"""

def render_q_and_a(code: str) -> AnswerSetter:
  w = widgets.HTML(value="")

  def _get_html(q: str, a: str):
    return f"""
    <table>
      <tr>
        <th style="border: 1px solid #999;">Code</th>
        <th style="border: 1px solid #999;">Explanation</th>
      </tr>
      <tr>
        <td style="padding: 2rem;">
          <code style="display: block; white-space: pre-wrap; word-wrap: break-word; background: black; padding: 1rem; text-align: left; line-height: 1.3;">{q}</code>
        </td>
        <td style="padding: 2rem; width: 60%;">
          {a}
        </td>
      </tr>
    </table>
    """

  def _answer_setter(answer: str):
    explanation_html = markdown.markdown(answer)
    w.value = _get_html(code, explanation_html)

  w.value = _get_html(code, spinner)
  display(w)

  return _answer_setter



code = """
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from tabulate import tabulate

# python
def greet(name):
    print(f"Hello, {name}!")
"""

answer = """
#python

foo bar
"""

# set_answer = render_q_and_a(code)
# sleep(2)
# set_answer(answer)

# New Section

## Run code explainer

In [31]:
def describe_code(code: str) -> str:
  prompt = f"""[INST] <<SYS>>
You are an expert programmer. Convert the code to fully funtioning bash code.
<</SYS>>
{code}[/INST]"""
#  prompt = f"""[INST] <<SYS>>
#You are an expert programmer. Convert the code to fully funtioning haskell code.
#<</SYS>>
#{code}[/INST]"""
  return llm(prompt)

In [32]:
sample_1 = """
tfidf_vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(stop_words=stop_words)
tfidf_matrix = tfidf_vectorizer.fit_transform([' '.join(tokenize(d)) for d in df['description']])
svd = SVD(10)
svd.fit(tfidf_matrix)

def cosine(a, b):
    eps = 1e-8
    if type(a) is np.ndarray:
        return a.dot(b) / ((np.linalg.norm(a) * np.linalg.norm(b)) + eps)
    else:
        return a.dot(b) / ((a.norm() * b.norm()) + eps)

def tfidf_model(query, document):
    query_vector = tfidf_vectorizer.transform([' '.join(tokenize(query, tfidf_vectorizer.get_feature_names()))]).todense()
    doc_vector = tfidf_vectorizer.transform([' '.join(tokenize(document, tfidf_vectorizer.get_feature_names()))]).todense()
    doc_vector = np.squeeze(np.asarray(doc_vector))
    query_vector = np.squeeze(np.asarray(query_vector))
    return cosine(query_vector, doc_vector)
"""

sample_2 = """

def lsa_model(query, document):
    query = ' '.join(tokenize(query, tfidf_vectorizer.get_feature_names()))
    document = ' '.join(tokenize(document, tfidf_vectorizer.get_feature_names()))
    query_vector = tfidf_vectorizer.transform([query]).todense()
    doc_vector = tfidf_vectorizer.transform([document]).todense()
    query_vector = svd.transform(query_vector)
    doc_vector = svd.transform(doc_vector)
    return cosine(np.squeeze(np.asarray(doc_vector)), np.squeeze(np.asarray(query_vector)))

def search(model, query, documents, names = None):
    scores = [model(query, document) for document in documents]
    ixs = list(reversed(np.argsort(scores)[-10:]))

    data = [[scores[i] for i in ixs], [documents[i][:150] for i in ixs]]
    if names is not None:
        data.append([names[i] for i in ixs])
    tabulate(data, header=False)
"""

sample_3 = """

def bubble_sort_list(ls):
    n = len(ls)
    for i in range(n):
        for j in range(0, n-i-1):
            if ls[j] > ls[j+1]:
                ls[j], ls[j+1] = ls[j+1], ls[j]
    return ls

# Example usage
ls = ["banana", "apple", "cherry", "date"]
lss = bubble_sort_list(ls)
print(lss)
"""

as_md(describe_code(sample_3))

  Certainly! Here is the bash code equivalent to the Python function `bubble_sort_list`:
```bash
#!/bin/bash

function bubble_sort_list {
    local ls
    # Get the list of strings to be sorted
    ls=($1)
    # Calculate the number of elements in the list
    n=${#ls[@]}
    # Sort the list using the bubble sort algorithm
    for ((i=0; i${j}==$j+1)) do
        # Compare adjacent elements in the list
        if [ "$ls[$j]" > "$ls[$j+1]" ]; then
            # Swap the elements if they are in the wrong order
            echo "Swapping ${ls[$j]} and ${ls[$j+1]}"
            ls[$j]="${ls[$j+1]}"
            ls[$j+1]="${ls[$j]}"
        fi
    done
    # Return the sorted list
    return ls
}
# Example usage
ls=("banana" "apple" "cherry" "date")
lss=$(bubble_sort_list "$ls")
echo "$lss"
```
Note that I've used bash syntax to define the function, and I've added a `do` keyword to wrap the loop that implements the bubble sort algorithm. I've also added an `else` clause to handle the case where two elements are in the wrong order.
I hope this helps! Let me know if you have any questions.